In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder\
                   .appName("DF Operations")\
                   .master("local[2]")\
                   .getOrCreate()

In [ ]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,FloatType,DateType,TimestampType,ArrayType

In [ ]:
custSchema = StructType([
                        StructField("cid",IntegerType()),
                        StructField("fname",StringType()),
                        StructField("lname",StringType()),
                        StructField("age",IntegerType()),
                        StructField("desig",StringType()) 
                      ])

In [ ]:
custDF = spark.read\
          .option("header",True)\
          .schema(custSchema)\
          .csv("d:\\data\\custs.txt")
custDF.show()

In [ ]:
def myconcat(fname,sep,lname):
    return(fname+sep+lname)

In [ ]:
myconcat("Shrikant","-","Gawande")

In [ ]:
custDF.show(3)

In [ ]:
from pyspark.sql.functions import col,lit

In [ ]:
from pyspark.sql.functions import udf

In [ ]:
myconcat_udf = udf(myconcat,returnType=StringType())

In [ ]:
custDF.withColumn("fullname",myconcat_udf(col("fname"),lit("-"),col("lname"))).show()

In [18]:
custDF.createOrReplaceTempView("cust_Table")

In [19]:
spark.sql("select desig , count(*) as cnt from \
          cust_table group by desig order by cnt desc limit 10").show()

+--------------------+---+
|               desig|cnt|
+--------------------+---+
|          Politician|227|
|Computer support ...|222|
|        Photographer|222|
|        Loan officer|221|
|           Librarian|218|
|         Firefighter|217|
|Computer software...|216|
|          Pharmacist|213|
|Human resources a...|212|
|       Social worker|212|
+--------------------+---+



In [20]:
spark.udf.register("myconcat_SQL",myconcat,returnType=StringType())

<function __main__.myconcat(fname, sep, lname)>

In [22]:
spark.sql("select fname,lname,myconcat_SQL(fname,'-',lname) from cust_Table").show()

+--------+----------+-----------------------------+
|   fname|     lname|myconcat_SQL(fname, -, lname)|
+--------+----------+-----------------------------+
|Kristina|     Chung|               Kristina-Chung|
|   Paige|      Chen|                   Paige-Chen|
|  Sherri|    Melton|                Sherri-Melton|
|Gretchen|      Hill|                Gretchen-Hill|
|   Karen|   Puckett|                Karen-Puckett|
| Patrick|      Song|                 Patrick-Song|
|   Elsie|  Hamilton|               Elsie-Hamilton|
|   Hazel|    Bender|                 Hazel-Bender|
| Malcolm|    Wagner|               Malcolm-Wagner|
| Dolores|McLaughlin|           Dolores-McLaughlin|
| Francis|  McNamara|             Francis-McNamara|
|   Sandy|    Raynor|                 Sandy-Raynor|
|  Marion|      Moon|                  Marion-Moon|
|    Beth|   Woodard|                 Beth-Woodard|
|   Julia|     Desai|                  Julia-Desai|
|  Jerome|   Wallace|               Jerome-Wallace|
|    Neal|  

In [25]:
for each in spark.catalog.listFunctions():
    if each.name == "myconcat_SQL":
        print(each)

Function(name='myconcat_SQL', description=None, className=None, isTemporary=True)
